In [1]:
!pip3 install -q gradio>=4.7.1
!pip3 install -q numpy>=1.26.2
!pip3 install -q pandas>=2.1.3
!pip3 install -q tensorflow>=2.15.0
!pip3 install -q transformers>=4.35.2
!pip3 install -q torch>=2.1.1
!pip3 install -q accelerate>=0.24.1
!pip3 install -q bitsandbytes>=0.41.2.post2
!pip3 install -q bitsandbytes-cuda113>=0.26.0.post2
!pip3 install -q langchain>=0.0.344

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 11.0.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.11.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.11.0 which is incompatible.
dask-cudf 23.8.0 requires dask==2023.7.1, b

In [2]:
import torch
import os
import copy
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,BitsAndBytesConfig
from langchain import PromptTemplate
import gradio as gr

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
model_name = '/kaggle/input/mistral/pytorch/7b-v0.1-hf/1'

In [4]:
class LoadLLM:
    model = None
    tokenizer = None
    pipe = None
    model_name = None
    def __init__(self, model_name="gpt2"):
        """
        loading the model
        :param model_name:
        """
        try:
            self.model_name = model_name
            if self.model_name is not None:
                self.load_model()
            else:
                return
            if self.model is not None:
                self.load_tokenizer()
            else:
                return
            if self.tokenizer is not None:
                self.load_pipeline()
            else:
                return
        except Exception as ex:
            print(f"Error while assigning model name or id: {ex}")
        return

    def load_model(self):
        try:
            self.model = AutoModelForCausalLM.from_pretrained(self.model_name, 
                                                              torch_dtype=torch.bfloat16, 
                                                              device_map="auto", 
                                                              trust_remote_code=True,)

            print("Successfully loaded the model")
        except Exception as ex:
            print(f"Error while loading model: {ex}")

    def load_tokenizer(self):
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            print("Successfully loaded the tokenizer")
        except Exception as ex:
            print(f"Error while loading tokenizer: {ex}")

    def load_pipeline(self):
        try:
            if self.model is not None and self.tokenizer is not None:
                self.pipe = pipeline("text-generation",
                                     model=self.model,
                                     tokenizer = self.tokenizer,
                                     torch_dtype=torch.bfloat16,
                                     device_map="auto")
                print("Successfully loaded the pipeline")
            else:
                print("Model or tokenizer not loaded")
        except Exception as ex:
            print(f"Error while loading the pipeline: {ex}")

    def get_pipeline(self):
        """:return: pipeline"""
        return self.pipe

    def get_tokenizer(self):
        """:return: tokenizer"""
        return self.tokenizer

    def get_model(self):
        """:return: model"""
        return self.model



    def get_output(self,
                   prompt,
                   max_length=256,
                   max_new_tokens=256,
                   temperature=0.2,
                   top_k=40,
                   top_p=0.2,
                   repetition_penalty=1.1,
                   num_beams=1,
                   no_repeat_ngram_size=0,
                   num_return_sequences=1,
                   stop_sequence=None,
                   do_sample=True,
                   model_name = "gpt2",
                   low_memory=True):
        try:
            stop_seq = None
            if self.model is None:
                return "Error while loading model, please fix your code"
            if self.tokenizer is None:
                return "Error while loading tokenizer, please fix your code"
            if self.pipe is None:
                return "Error while loading pipeline, please fix your code"
            print("Successfully loaded pipeline")
            num_beams = int(num_beams)
            if stop_sequence is not None and type(stop_sequence) == list and len(stop_sequence) > 0:
                text_encodings = []
                for word in stop_sequence:
                    if type(word) == str:
                        words = word.strip().split()
                        if len(words) == 1 and len(words[0].strip()) > 0:
                            text_encodings.append([words[0].strip()])
                        elif len(words) > 1:
                            for w in words:
                                w = w.strip()
                                if len(w) > 0:
                                    text_encodings.append([w])
                #             print(text_encodings)
                if len(text_encodings) > 0:
                    stop_seq = self.tokenizer(text_encodings,
                                         padding="max_length", is_split_into_words=True, truncation=True,
                                         return_tensors="pt")

                #         print(stop_seq)
            output = self.pipe(prompt, num_beams=num_beams,
                          do_sample=do_sample,
                          max_length=max_length,
                          max_new_tokens=max_new_tokens,
                          temperature=temperature,
                          top_k=top_k,
                          top_p=top_p,
                          repetition_penalty=repetition_penalty,
                          no_repeat_ngram_size=no_repeat_ngram_size,
                          # fix the stop_sequence to stoping_criteria so it would stop post that token.
                          #                       stop_sequence = stop_seq,
                          num_return_sequences=num_return_sequences,
                          low_memory=low_memory)
            if output:
                output = output[0]["generated_text"]
            return output
        except Exception as ex:
            print(f"Error while running llm: {ex}")
        return


In [5]:
model = LoadLLM(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Successfully loaded the model
Successfully loaded the tokenizer
Successfully loaded the pipeline


In [6]:
%%time
prompt_job = "Generate a spellbinding Job Description as a Hiring Manager using only the input Job Title,"\
" Experience and Skills. Don't use Price details, HTML tags, Javascript, Programming languages, URLs, "\
"or information not in the input Keywords to generate the Job Description.\nKeywords: Machine Learning Engineer"\
" - NLP, 3+ years of experience, Natural Language Processing(NLP), LSTM, RNN, Neural nets, LLMs, ML algorithms,"\
" Data Structures, Algorithms, Python programming\n Description:"
job_output = model.get_output(prompt_job,
                                num_beams=3,
                                max_length=512, 
                                max_new_tokens=512, 
                                top_k=512, 
                                temperature = 0.3, 
                                top_p=0.5, 
                                no_repeat_ngram_size=0)
print(job_output)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Successfully loaded pipeline


Both `max_new_tokens` (=512) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generate a spellbinding Job Description as a Hiring Manager using only the input Job Title, Experience and Skills. Don't use Price details, HTML tags, Javascript, Programming languages, URLs, or information not in the input Keywords to generate the Job Description.
Keywords: Machine Learning Engineer - NLP, 3+ years of experience, Natural Language Processing(NLP), LSTM, RNN, Neural nets, LLMs, ML algorithms, Data Structures, Algorithms, Python programming
 Description:

Machine Learning Engineer - NLP

We are looking for a Machine Learning Engineer with experience in Natural Language Processing (NLP) to join our team. You will be responsible for developing and implementing machine learning models to extract insights from text data. You will work with a team of data scientists, software engineers, and product managers to build and deploy machine learning models that can be used to improve our products and services.

Responsibilities:

- Develop and implement machine learning models to e

In [10]:
gr.close_all()
with gr.Blocks() as demo:
    print("Entered the demo")
    gr.Markdown("# LLM with Gradio")
    template = gr.Textbox(label="template",info="Input template wrt which you want to generate the Description", lines=5)
    print(f"Input template to gradio: {template}")
    # # data = gr.Textbox(label="data", info="Your data as dictionary")
    # # logging.info(f"Input data to gradio: {data}")
    btn = gr.Button("Submit") #Submit button side by side!
    with gr.Accordion("Advanced options", open=False): #Let's hide the advanced options!
            with gr.Row():
                with gr.Column():
                    max_tokens = gr.Slider(label="max_length", minimum=0, maximum=1024, value=256, info="Max output tokens you want to generate")
                    temperature = gr.Slider(label="temperature", minimum=0.00, maximum=0.99, value=0.2,
                                            info="Temperature to experiment with based on creativity")
                    top_k = gr.Slider(label="top_k", minimum=0, maximum=512, value=50,
                                      info="Enter last k words to give attention while predicting current word. "\
                                      "Top-K sampling involves sorting the probability distribution and selecting "
                                      "the top-K most likely candidates. By adjusting the value of K, you can control "\
                                      "the diversity of the generated text. A smaller K results in more deterministic and focused outputs, "\
                                      "while a larger K introduces more randomness.")

                    # num_beams = int(round(gr.Slider(label="num_beams", minimum=0, maximum=5, value=1,
                    #                                 info="Number of beams for beam search. It should be 1,2, etc not a float" )))

                    num_beams = gr.Number(label="num_beams",
                                          minimum=0, maximum=5,
                                          value=1,
                                          info="Number of beams for beam search." )
                with gr.Column():
                    max_new_tokens = gr.Slider(label="max_new_tokens", minimum=0, maximum=1024, value=256,
                                               info="Max new tokens you want to generate")
                    repeat_penalty = gr.Slider(label="repeat_penalty", minimum=0.00, maximum=5.0, value=1.1,
                                               info="Enter repeat penatlity")
                    top_p = gr.Slider(label="top_p", minimum=0.00, maximum=1.0, value=0.2,
                                      info="Adjusting the value of  p allows you to control the diversity of the "\
                                      "generated text. A smaller p results in more deterministic and focused outputs, "\
                                      "while a larger p introduces more randomness.")
            with gr.Row():
                stop_words = gr.Textbox(label="stop words", info="Please enter as list example to stop at strings 'Keywords:' and '####' : ['Keywords:', '####']")


    output = gr.Textbox(label="Generated text") #Move the output up too
    # btn.click(fn=model_info.get_output, inputs=[template], outputs=[output])
    # num_beams = int(num_beams)
    btn.click(fn=model.get_output,
              inputs=[template,
                      max_tokens, max_new_tokens,
                      temperature,
                      top_k,
                      top_p,
                      repeat_penalty,
                      num_beams],
              outputs=[output])



gr.close_all()


Entered the demo
Input template to gradio: textbox


In [11]:
demo.launch(share=True)#, server_port=8181)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://9b159e67775a41d63c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
